# Effecient Multi-Party Private Set Intersection

### By Malia Kency and John Owens

First, load imports

In [1]:
import  players
import  commitment_scheme as CM
import  bloom_filter as BF
import  security_params as SP
import  math

Next, set constant protocol parameters then calculate the others

   * NumPlayer = Total number of players Pi
   * PlayerInputSize = Size of the player's input sets
   * Nbf = Length of the Bloom Filter
   * k = Number of hash functions to use
   * SecParam = Security paremeter. 40 in the paper's example
   * Nmaxones = Maximum number of ones allowed after cut-and-choose
   * p = percentage of total 1s to be used for cut-and-choose
   * a = sampling weight of 1s vs. 0s for all Pi

In [2]:
NumPlayers = 3 
PlayerInputSize = 3
Nbf = 3
k = BF.GetNumHashFuncs(Nbf, PlayerInputSize)
SecParam = 20
Nmaxones = 1
p = 0.25
a = 0.5

# Generate the other parameters: Not, m1h, check values gamma and gammaStar
Protocol_Params = SP.Security_Params(Nmaxones, PlayerInputSize, k, p, a, SecParam, Nbf)

After parameters are set, we can simluate the first major step of the protocol: Random OT

Note: The number of Oblivious Transfers that occur is determined by Not, which is the result of several calculations in security_parameters.py

In [7]:
players = players.Players(NumPlayers)
# scheme = CM.CommitmentScheme()
# players.broadcast("test")
players.perform_random_ot(Protocol_Params.a, Protocol_Params.Not)
print("Player 0 has " + str( players.players[0].getNumberOfOnes() ) + " ones. Should be: " + str( Protocol_Params.Not * (NumPlayers - 1) ) )
print("Player 1 has " + str( players.players[1].getNumberOfOnes() ) + " ones. Ideal: " + str( Protocol_Params.Not * Protocol_Params.a ))
print("Player 2 has " + str( players.players[2].getNumberOfOnes() ) + " ones. Ideal: " + str( Protocol_Params.Not * Protocol_Params.a ))
print("\'a\' defined as " + str( Protocol_Params.a ));

Player 0 created
Player 1 created
Player 2 created
Performing random OT 1 of 8
Performing random OT 2 of 8
Performing random OT 3 of 8
Performing random OT 4 of 8
Performing random OT 5 of 8
Performing random OT 6 of 8
Performing random OT 7 of 8
Performing random OT 8 of 8
Player 0 has 16 ones. Should be: 16
Player 1 has 5 ones. Ideal: 4.0
Player 2 has 2 ones. Ideal: 4.0
'a' defined as 0.5
